# 0. Информация об видеокарте

In [1]:
!nvidia-smi

Sat Apr 13 22:22:48 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.12                 Driver Version: 546.12       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080      WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   48C    P5              49W / 370W |   1828MiB / 10240MiB |     45%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# 1. Загрузка библиотек и объявление путей

In [2]:
import os
import glob
import random
import cv2
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import json
from pprint import pprint
import shutil
import yaml
from ultralytics import YOLO

import warnings
warnings.filterwarnings("ignore")

In [3]:
# Пути до датасетов с автомобилями
PATH_CAR = './DATASET/CarDetection' # Путь до датасета Car Detection
PATH_CARS = './DATASET/CarsDetection/Cars Detection/' # Путь до датасета Cars Detection
PATH_CAR_PERSON = './DATASET/CarPerson/Car-Person-v2-Roboflow-Owais-Ahmad/' # Путь до датасета Car Person

# Пути до датасетов с ЛЭП
PATH_STN = './DATASET/STN' # Путь до датасета STN PLAD
PATH_UTILLUTY_POLE = './DATASET/UtilityPole/' # Путь до датасета Utiluty Pole
PATH_TRANSMISSION = './DATASET/Transmission/data_original_size_v1/data_original_size/' # Путь до датасета Transmission

# Пути до датасетов с деревьями
PATH_TREE = './DATASET/Tree/' # Путь до датасета Tree Detection
PATH_TREE_DETECTION = './DATASET/TreeDetection/' # Путь до датасета Tree Detection
PATH_TREE_100 = './DATASET/Trees100/' # Путь до датасета Tree 100

# Пути до датасетов с птицами
PATH_BIRD_ANNOTATION = './DATASET/BirdAnnotation/' # Путь до датасета Bird Annotation 
PATH_BIRDS_DETECTOR = './DATASET/BirdsDetector/' # Путь до датасета Birds Detector
PATH_BIRDS_V3 = './DATASET/BirdsV3/' # Путь до датасета BirdsV3

# Пути до датасетов со зданиями
PATH_BUILDING_DETECTION = './DATASET/BuildingDetection/' # Путь до датасета Building Detection
PATH_JOSH_EDITS = './DATASET/JoshEdits/' # Путь до датасета JoshEdits

# Словарь по понижению размрености датасетов
DATASET_DOWNGRADES = {
    'dataCarDetection': 2, 
    'dataCarPerson': 3, 
    'dataCarsDetection': 4,
    'dataSTN': 1, 
    'dataUnilityPole': 1, 
    'dataTransmission': 1,
    'dataTree': 2, 
    'dataTreeDetection': 2, 
    'dataTree100': 2,
    'dataBirdAnnotation': 4, 
    'dataBirdsDetectod': 4, 
    'dataBirdsV3': 2,
    'dataBuildingDetection': 2, 
    'dataJoshEdits': 2
}

In [4]:
def print_variable_name(variable):
    '''
        Функция позволяет получить имя переменной
        Input: 
            variable - переменная
        Output:
            variable_name - имя переменной
    '''
    variable_name = [name for name, value in globals().items() if value is variable][-1]
    return variable_name

# 2. Обработка данных

In [5]:
def converterYoloToDataFrame(labels_list, path, classes):
    '''
        Функция позволяет перенести все данные с формата YOLO в DataFrame
        Input:
            labels_list - набор названий изображений
            path - путь до папки с изображениями
    '''
    labels = {} # Словарь объктов
    number_item = 0 # Номер записи
    for item in tqdm(labels_list):
        # Читаем все записи
        with open(path + item) as f:
            lines = f.readlines()
    
        # Проходимся по каждой записи
        for line in lines:
            # Получаем координаты в процентах
            x_center, y_center, w, h = (float(number) for number in line.split()[1:]) 
    
            # Заносим в словарь
            labels[number_item] = {
                'image_name': '/'.join(path.split('/')[:-2]) + '/images/' + item[:-3] + 'jpg',
                'x_center': x_center, 
                'y_center': y_center, 
                'w': w, 
                'h': h,
                'classes': classes
            }
            number_item += 1
    return labels

In [6]:
def datasetMinimization(dataSet, count):
    '''
        Функция позволяет понизить размер DataFrame за счёт удаления фотографий
        Input:
            dataSet - первоначальный DataFrame
            count - количество, во сколько раз сжать DataFrame
    '''
    # Список во всеми именами фотографий
    list_image_name = dataSet['image_name'].unique().tolist()

    # Кол-во фотографий в начале
    start_count_image = len(list_image_name)
    
    # Получаем именя изображений, которые оставляем
    name_file = random.sample(list_image_name, len(list_image_name) // count)

    # Записи с этими файлами
    dataSet = dataSet[dataSet['image_name'].isin(name_file)]

    # Кол-во фотографий после обработки
    end_count_image = len(dataSet['image_name'].unique())
    
    # Выведем системноо сообщение об результате сокращения фотографий
    print(f'There were {start_count_image} photos, now there are {end_count_image}')
    
    return dataSet

## 2.0 Классы для модели
    0 - машина
    1 - ЛЭП
    2 - Деревья
    3 - Птицы
    4 - Дом/здание
    5 - Провода

## 2.1. Датасеты с автомобилями
### 2.1.1 Обработка датасета Car Detection

In [7]:
img_h, img_w = (380, 676) # Размеры изображения
dataCarDetection = pd.read_csv(PATH_CAR + '/train_solution_bounding_boxes (1).csv') # Читаем датасет

dataCarDetection.rename(columns={'image':'image_name'}, inplace=True) # Переименовываем колонку

dataCarDetection.sample(10) # посмотрим на данные

,image_name,xmin,ymin,xmax,ymax
123,vid_4_16120.jpg,451.971056,181.655405,581.594790,228.605212
414,vid_4_600.jpg,286.639653,187.524131,407.947902,232.028636
264,vid_4_21560.jpg,374.196816,187.524131,501.374819,226.648970
444,vid_4_6300.jpg,0.000000,192.903797,46.468886,234.473938
360,vid_4_29460.jpg,545.887120,176.275740,630.509407,212.955277
269,vid_4_21600.jpg,0.000000,197.794401,48.425470,233.281853
312,vid_4_26380.jpg,603.606368,185.567889,665.238784,207.575611
146,vid_4_17240.jpg,421.154848,185.567889,578.659913,248.167632
21,vid_4_11380.jpg,514.581766,176.275740,615.835022,229.094273
458,vid_4_6420.jpg,612.410999,179.699163,676.000000,212.466216


In [8]:
# Получим координаты центра объекта в процентах 
dataCarDetection['x_center'] = (dataCarDetection['xmin'] + dataCarDetection['xmax']) / 2 / img_w 
dataCarDetection['y_center'] = (dataCarDetection['ymin'] + dataCarDetection['ymax']) / 2 / img_h 

# Получим размеры объекта в процентах
dataCarDetection['w'] = (dataCarDetection['xmax'] - dataCarDetection['xmin']) / img_w # Получаем ширину объекта в процентах
dataCarDetection['h'] = (dataCarDetection['ymax'] - dataCarDetection['ymin']) / img_h # Получаем ширину объекта в процентах
dataCarDetection['classes'] = 0 # Задаём класс объекта
dataCarDetection.drop(
    ['xmin', 'xmax', 'ymin', 'ymax'], # Удаляем ненужные признаки
    axis=1, inplace=True
)

# Изменим путь до файла
dataCarDetection['image_name'] = f'{PATH_CAR}/training_images/' + dataCarDetection['image_name']

# Очистим память 
del img_h, img_w

# Уменьшаем размер DataFrame
dataCarDetection = datasetMinimization(dataCarDetection, DATASET_DOWNGRADES[print_variable_name(dataCarDetection)])

dataCarDetection.sample(10) # Посмотрим на данные

There were 355 photos, now there are 177


,image_name,x_center,y_center,w,h,classes
541,./DATASET/CarDetection/training_images/vid_4_9...,0.612891,0.548454,0.156591,0.089378,0
507,./DATASET/CarDetection/training_images/vid_4_9...,0.138019,0.540682,0.085215,0.058290,0
140,./DATASET/CarDetection/training_images/vid_4_1...,0.820188,0.549469,0.162808,0.124839,0
85,./DATASET/CarDetection/training_images/vid_4_1...,0.259768,0.518581,0.060781,0.063063,0
325,./DATASET/CarDetection/training_images/vid_4_2...,0.625543,0.547539,0.181621,0.105534,0
306,./DATASET/CarDetection/training_images/vid_4_2...,0.805716,0.529521,0.122287,0.079794,0
257,./DATASET/CarDetection/training_images/vid_4_2...,0.803184,0.528877,0.178003,0.119691,0
24,./DATASET/CarDetection/training_images/vid_4_1...,0.054993,0.572635,0.109986,0.096525,0
53,./DATASET/CarDetection/training_images/vid_4_1...,0.370839,0.526303,0.062952,0.068211,0
408,./DATASET/CarDetection/training_images/vid_4_3...,0.542692,0.557835,0.180897,0.136422,0


In [9]:
import inspect
xer = 0

print_variable_name(dataCarDetection)

'dataCarDetection'

## 2.1.2. Обработка датасета Cars Detection

In [10]:
dataCarsDetection = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_CARS}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_CARS}{sample}/labels/', 0)) # Получаем датасет с разметкой
    dataCarsDetection = pd.concat([dataCarsDetection, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataCarsDetection = datasetMinimization(dataCarsDetection, DATASET_DOWNGRADES[print_variable_name(dataCarsDetection)])

dataCarsDetection.sample(10) # Посмотрим на данные

  0%|          | 0/878 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/126 [00:00<?, ?it/s]

There were 1254 photos, now there are 313


,image_name,x_center,y_center,w,h,classes
56,./DATASET/CarsDetection/Cars Detection/train/i...,0.503606,0.606971,0.896635,0.715144,0
1625,./DATASET/CarsDetection/Cars Detection/train/i...,0.986779,0.326923,0.021635,0.197115,0
202,./DATASET/CarsDetection/Cars Detection/train/i...,0.5,0.564904,0.995192,0.605769,0
715,./DATASET/CarsDetection/Cars Detection/train/i...,0.814904,0.388221,0.365385,0.362981,0
21,./DATASET/CarsDetection/Cars Detection/test/im...,0.700721,0.813702,0.182692,0.106971,0
1213,./DATASET/CarsDetection/Cars Detection/train/i...,0.947115,0.358173,0.100962,0.245192,0
190,./DATASET/CarsDetection/Cars Detection/valid/i...,0.695913,0.449519,0.539663,0.609375,0
1470,./DATASET/CarsDetection/Cars Detection/train/i...,0.539663,0.532452,0.760817,0.444712,0
1047,./DATASET/CarsDetection/Cars Detection/train/i...,0.923077,0.378606,0.126202,0.225962,0
1279,./DATASET/CarsDetection/Cars Detection/train/i...,0.461538,0.510817,0.594952,0.58774,0


### 2.1.3. Обработка датасета Car Person

In [11]:
dataCarPerson = pd.DataFrame(columns=dataCarDetection.columns)
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_CAR_PERSON}{sample}/labels') # Список файлов с разметкой
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_CAR_PERSON}{sample}/labels/', 0)) # Получаем датасет с разметкой
    dataCarPerson = pd.concat([dataCarPerson, labels.T]) # Соединяем выборки в единый датасет
    
del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataCarPerson = datasetMinimization(dataCarPerson, DATASET_DOWNGRADES[print_variable_name(dataCarPerson)])

dataCarPerson.sample(10) # Посмотрим на данные

  0%|          | 0/1571 [00:00<?, ?it/s]

  0%|          | 0/448 [00:00<?, ?it/s]

  0%|          | 0/224 [00:00<?, ?it/s]

There were 2243 photos, now there are 747


,image_name,x_center,y_center,w,h,classes
5825,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.784856,0.233173,0.105769,0.302885,0
3249,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.825721,0.628606,0.084135,0.248798,0
7820,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.114183,0.382212,0.228365,0.247596,0
4508,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.945913,0.688702,0.03726,0.079327,0
8970,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.036058,0.526442,0.069712,0.788462,0
3544,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.586538,0.061298,0.066106,0.046875,0
1062,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.126202,0.288462,0.100962,0.346154,0
5987,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.778846,0.300481,0.090144,0.274038,0
9497,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.490385,0.623798,0.225962,0.75,0
6171,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.168269,0.524038,0.231971,0.323317,0


## 2.2. Обработка датасет с ЛЭП
### 2.2.1 Обработка датасета STN_PLAD

In [12]:
# Читаем файл с разметкой датасета STN
with open(PATH_STN + '/annotations.json') as f:
    dataSTN = json.load(f)

pprint(dataSTN['categories']) # Посмотрим на категории

# Получим два DataFrame, один с изображениями, другой с разметкой объектов
data = pd.DataFrame(dataSTN['images']).rename(columns={'id': 'image_id'})
data1 =  pd.DataFrame(dataSTN['annotations'])

# Объединим два дасатсета и удалим лишние признаки
dataSTN = pd.merge(data, data1, on='image_id', how='left').drop(
    [
        'licence', 'segmentation', 'date_captured', 'coco_url', 
        'iscrowd', 'image_id'
    ], axis=1
)

# Очистим память 
del data, data1 

dataSTN.sample(10) # Посмотрим на данные

[{'id': 0, 'name': 'tower', 'supercategory': 'tower'},
 {'id': 1, 'name': 'insulator', 'supercategory': 'component'},
 {'id': 2, 'name': 'spacer', 'supercategory': 'component'},
 {'id': 3, 'name': 'damper', 'supercategory': 'component'},
 {'id': 5, 'name': 'plate', 'supercategory': 'component'}]


,file_name,height,width,flickr_url,area,bbox,category_id,id
1370,20181127-A1/DJI_0497.JPG,3078,5472,,1276,"[1444, 1879, 29, 44]",3,1377
1109,20181129-R/DJI_0016.JPG,3648,5472,,559,"[2495, 2558, 43, 13]",3,1115
943,20181129-R/DJI_0019.JPG,3648,5472,,2491,"[2768, 1083, 53, 47]",5,947
1320,20181127-A1/DJI_0439.JPG,3078,5472,,770,"[2161, 1671, 55, 14]",3,1327
2253,20181127-A1/DJI_0516.JPG,3078,5472,,1800,"[3713, 1299, 75, 24]",3,2266
1151,20181127-A1/DJI_0467.JPG,3078,5472,,1680,"[2810, 2478, 24, 70]",3,1157
1357,20181127-A1/DJI_0497.JPG,3078,5472,,1440,"[4174, 1786, 30, 48]",3,1364
318,20181127-A1/DJI_0438.JPG,3078,5472,,29189,"[2487, 2422, 101, 289]",2,319
1043,20181127-A1/DJI_0417.JPG,3078,5472,,1755,"[4797, 1338, 65, 27]",3,1049
1226,20181129-R/DJI_0003.JPG,3648,5472,,105968,"[841, 1403, 179, 592]",1,1232


In [13]:
# Получим координаты объекта
dataSTN[['xmin', 'ymin', 'xmax', 'ymax']] = dataSTN['bbox'].apply(lambda x: pd.Series(x))
dataSTN['xmax'] = dataSTN['xmax'] + dataSTN['xmin']
dataSTN['ymax'] = dataSTN['ymax'] + dataSTN['ymin']

# Получим координаты центра объекта в процентах 
dataSTN['x_center'] = (dataSTN['xmin'] + dataSTN['xmax']) / 2 / dataSTN['width']
dataSTN['y_center'] = (dataSTN['ymin'] + dataSTN['ymax']) / 2 / dataSTN['height']

# Получим размеры объекта в процентах
dataSTN['w'] = (dataSTN['xmax'] - dataSTN['xmin']) / dataSTN['width']
dataSTN['h'] = (dataSTN['ymax'] - dataSTN['ymin']) / dataSTN['height']

# Переименуем признаки
dataSTN.rename(
    columns={
        'file_name': 'image_name',
        'category_id': 'classes'
    }, 
    inplace=True
) 

# Удалим ненужные признаки
dataSTN.drop(
    [
        'height', 'width', 'flickr_url',
        'area', 'bbox', 'id', 'xmin', 
        'xmax', 'ymin', 'ymax'
    ], 
    axis=1, inplace=True
)

# Изменим путь до файла
dataSTN['image_name'] = './DATASET/STN/' + dataSTN['image_name']

# Оставим фотографии принадлижащие только к опоре ЛЭП
dataSTN = dataSTN[dataSTN['classes'] == 0]
dataSTN['classes'] = dataSTN['classes'] + 1

dataSTN.sample(10) # Посомтрим на данные

,image_name,classes,x_center,y_center,w,h
1251,./DATASET/STN/20181127-A1/DJI_0469.JPG,1,0.884594,0.411144,0.230811,0.643600
440,./DATASET/STN/20181129-R/DJI_0020.JPG,1,0.123721,0.415707,0.016447,0.083059
2351,./DATASET/STN/20181129-R/DJI_0023.JPG,1,0.237756,0.384731,0.018640,0.039748
205,./DATASET/STN/20181127-A1/DJI_0462.JPG,1,0.451937,0.569688,0.126827,0.860624
575,./DATASET/STN/20181129-R/DJI_0010.JPG,1,0.468750,0.675713,0.247076,0.648575
1286,./DATASET/STN/20181127-A1/DJI_0430.JPG,1,0.378289,0.317414,0.033626,0.176738
1118,./DATASET/STN/20181129-R/DJI_0016.JPG,1,0.474872,0.461212,0.013341,0.109375
2051,./DATASET/STN/20181127-A1/DJI_0416.JPG,1,0.586897,0.500162,0.173063,0.999675
1582,./DATASET/STN/20181127-A1/DJI_0486.JPG,1,0.478710,0.176251,0.040753,0.143925
2061,./DATASET/STN/20181127-A1/DJI_0476.JPG,1,0.507858,0.659519,0.614035,0.680962


### 2.2.2. Обработка датасета Utility Pole

In [14]:
dataUnilityPole = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'test']:
    labels_name = os.listdir(f'{PATH_UTILLUTY_POLE}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_UTILLUTY_POLE}{sample}/labels/', 1)) # Получаем датасет с разметкой
    dataUnilityPole = pd.concat([dataUnilityPole, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память
dataUnilityPole.sample(10) # Посмотрим на данные

  0%|          | 0/179 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

,image_name,x_center,y_center,w,h,classes
89,./DATASET/UtilityPole/train/images/distributio...,0.3125,0.776442,0.072115,0.259615,1
150,./DATASET/UtilityPole/train/images/images--7-_...,0.545673,0.501202,0.254808,0.949519,1
92,./DATASET/UtilityPole/train/images/distributio...,0.6875,0.776442,0.072115,0.259615,1
49,./DATASET/UtilityPole/train/images/a93a35ee792...,0.223558,0.629808,0.091346,0.307692,1
231,./DATASET/UtilityPole/train/images/Products_Po...,0.497596,0.473558,0.144231,0.927885,1
249,./DATASET/UtilityPole/train/images/twup_jpg.rf...,0.715144,0.49399,0.252404,0.959135,1
60,./DATASET/UtilityPole/train/images/b3978835746...,0.270433,0.481971,0.266827,0.742788,1
270,./DATASET/UtilityPole/train/images/Wood-Produc...,0.717548,0.710337,0.055288,0.189904,1
208,./DATASET/UtilityPole/train/images/lws-elam-1_...,0.549279,0.469952,0.310096,0.901442,1
41,./DATASET/UtilityPole/train/images/a93a35ee792...,0.223558,0.629808,0.091346,0.307692,1


### 2.2.3. Обработка датасета Transmission

In [15]:
list_image = glob.glob(PATH_TRANSMISSION + '*.jpg')
labels = {} # Словарь объктов
number_item = 0 # Номер записи
for path_image in tqdm(list_image):
    with open(path_image[:-3] + 'json') as f:
        json_file = json.load(f)
    xmax = 0
    xmin = 2**10
    ymax = 0
    ymin = 2**10
    height = json_file['imageHeight']
    width = json_file['imageWidth']
    for item in json_file['shapes']:
        if item['label'] in ['tower_wooden', 'cable']:
            for data in item['points']:
                x, y = data
                xmax = x if x > xmax else xmax
                ymax = y if x > ymax else ymax
                xmin = x if y < xmin else xmin
                ymin = y if y < ymin else ymin
            x_center = (xmin + xmax) / 2 / width
            y_center = (ymin + ymax) / 2 / height
            w = (xmax - xmin) / width
            h = (ymax - ymin) / height
            labels[number_item] = {
                'image_name': path_image,
                'x_center': 0.0 if x_center < 0.0 else 1.0 if x_center > 1.0 else x_center, 
                'y_center': 0.0 if y_center < 0.0 else 1.0 if y_center > 1.0 else x_center, 
                'w': 0.0 if w < 0.0 else 1.0 if w > 1.0 else w, 
                'h': 0.0 if h < 0.0 else 1.0 if h > 1.0 else h,
                'classes': 1 if item['label'] == 'tower_wooden' else 5
            }
            number_item += 1
            
dataTransmission = pd.DataFrame(labels).T # Переносим все в DataFrame

# Очищаем память
del list_image, labels, number_item, xmin, xmax, ymin, ymax, h, w, json_file 

# Уменьшаем размер DataFrame
dataTransmission = datasetMinimization(dataTransmission, DATASET_DOWNGRADES[print_variable_name(dataTransmission)])

dataTransmission.sample(10) # Посмотрим на данные

  0%|          | 0/1242 [00:00<?, ?it/s]

There were 1231 photos, now there are 1231


,image_name,x_center,y_center,w,h,classes
5315,./DATASET/Transmission/data_original_size_v1/d...,0.499744,0.499744,0.999169,0.042036,5
5132,./DATASET/Transmission/data_original_size_v1/d...,0.471311,0.471311,0.942623,0.688626,5
9018,./DATASET/Transmission/data_original_size_v1/d...,0.49987,0.49987,0.99974,0.498279,5
5654,./DATASET/Transmission/data_original_size_v1/d...,0.49987,0.49987,0.99974,0.210317,5
4005,./DATASET/Transmission/data_original_size_v1/d...,0.027063,0.027063,0.0,0.420479,5
5037,./DATASET/Transmission/data_original_size_v1/d...,0.87124,0.87124,0.079834,0.810275,5
8663,./DATASET/Transmission/data_original_size_v1/d...,0.503611,0.503611,0.806858,0.212508,5
6449,./DATASET/Transmission/data_original_size_v1/d...,0.576811,0.576811,0.845857,0.385522,5
4470,./DATASET/Transmission/data_original_size_v1/d...,0.608887,0.608887,0.781704,0.901265,5
2128,./DATASET/Transmission/data_original_size_v1/d...,0.748679,0.748679,0.007419,0.514403,5


## 2.3. Обработка датасетов с деревьями
### 2.3.1. Обработка датасета Tree Object Detecion

In [16]:
dataTree = pd.DataFrame(columns=dataCarDetection.columns)
labels_name = os.listdir(f'{PATH_TREE}train/labels/') # Список файлов с разметкой
labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_TREE}train/labels/', 2)) # Получаем датасет с разметкой
dataTree = pd.concat([dataTree, labels.T]) # Соединяем выборки в единый датасет
    
del labels_name, labels # Очищаем память

dataTree.sample(10) # Посмотрим на данные

  0%|          | 0/214 [00:00<?, ?it/s]

,image_name,x_center,y_center,w,h,classes
247,./DATASET/Tree/train/images/images-13-_jpg.rf....,0.514062,0.426563,0.971875,0.685937,2
196,./DATASET/Tree/train/images/big-tree-pixabay_j...,0.225781,0.860156,0.021875,0.059375,2
640,./DATASET/Tree/train/images/tree02_jpg.rf.0af1...,0.891406,0.302344,0.076563,0.604688,2
263,./DATASET/Tree/train/images/images-25-_jpg.rf....,0.850781,0.376563,0.117188,0.26875,2
97,./DATASET/Tree/train/images/366_jpg.rf.5b248cb...,0.5125,0.525,0.6375,0.945312,2
414,./DATASET/Tree/train/images/istockphoto-160910...,0.516406,0.514062,0.710938,0.801562,2
453,./DATASET/Tree/train/images/mqdefault_jpg.rf.6...,0.071875,0.552344,0.129688,0.279687,2
111,./DATASET/Tree/train/images/3888_jpg.rf.e3627a...,0.971094,0.822656,0.040625,0.06875,2
68,./DATASET/Tree/train/images/20230709_155608_jp...,0.5,0.354687,0.317188,0.548438,2
594,./DATASET/Tree/train/images/the-difference-bet...,0.811719,0.5875,0.060937,0.071875,2


### 2.3.2. Обработка датасета TreeDetection 

In [17]:
dataTreeDetection = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_TREE_DETECTION}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_TREE_DETECTION}{sample}/labels/', 2)) # Получаем датасет с разметкой
    dataTreeDetection = pd.concat([dataTreeDetection, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataTreeDetection = datasetMinimization(dataTreeDetection, DATASET_DOWNGRADES[print_variable_name(dataTreeDetection)])

dataTreeDetection.sample(10) # Посмотрим на данные

  0%|          | 0/1803 [00:00<?, ?it/s]

  0%|          | 0/174 [00:00<?, ?it/s]

  0%|          | 0/86 [00:00<?, ?it/s]

There were 2060 photos, now there are 1030


,image_name,x_center,y_center,w,h,classes
911,./DATASET/TreeDetection/train/images/211103_11...,0.134375,0.460156,0.221875,0.536719,2
75,./DATASET/TreeDetection/test/images/230810_171...,0.484375,0.485156,0.924219,0.6125,2
1313,./DATASET/TreeDetection/train/images/230529_17...,0.244531,0.519531,0.407031,0.44375,2
454,./DATASET/TreeDetection/train/images/210421_16...,0.5,0.467969,0.721094,0.796875,2
1764,./DATASET/TreeDetection/train/images/230930_17...,0.485938,0.553906,0.653906,0.865625,2
154,./DATASET/TreeDetection/train/images/14e1d8b4-...,0.525,0.586719,0.921875,0.56875,2
1021,./DATASET/TreeDetection/train/images/220220_17...,0.316406,0.5,0.632812,1.0,2
171,./DATASET/TreeDetection/train/images/17bf0560-...,0.489844,0.414844,0.801562,0.74375,2
2631,./DATASET/TreeDetection/train/images/ab2f3ffa-...,0.30625,0.566406,0.185156,0.403125,2
1547,./DATASET/TreeDetection/train/images/230814_17...,0.728906,0.589844,0.5125,0.76875,2


### 2.3.3. Обработка датасета Tree 100

In [18]:
dataTree100 = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_TREE_100}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_TREE_100}{sample}/labels/', 2)) # Получаем датасет с разметкой
    dataTree100 = pd.concat([dataTree100, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

dataTree100.sample(10) # Посмотрим на данные

  0%|          | 0/82 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

,image_name,x_center,y_center,w,h,classes
140,./DATASET/Trees100/train/images/arvore82_jpg.r...,0.5625,0.348438,0.1,0.239063,2
137,./DATASET/Trees100/train/images/arvore82_jpg.r...,0.248438,0.288281,0.136719,0.484375,2
2,./DATASET/Trees100/train/images/arvore01_jpg.r...,0.94375,0.871875,0.092188,0.082031,2
26,./DATASET/Trees100/valid/images/arvore57_jpg.r...,0.546094,0.530469,0.535937,0.617188,2
6,./DATASET/Trees100/test/images/arvore118_jpg.r...,0.107813,0.278906,0.210156,0.360938,2
23,./DATASET/Trees100/valid/images/arvore46_jpg.r...,0.496875,0.352344,0.217188,0.315625,2
143,./DATASET/Trees100/train/images/arvore82_jpg.r...,0.741406,0.467969,0.046875,0.110937,2
10,./DATASET/Trees100/test/images/arvore121_jpg.r...,0.938281,0.627344,0.123438,0.439844,2
162,./DATASET/Trees100/train/images/arvore92_jpg.r...,0.289062,0.421875,0.486719,0.55,2
126,./DATASET/Trees100/train/images/arvore67_jpg.r...,0.285156,0.313281,0.348438,0.426563,2


## 2.4. Обработка датасета с птицами
### 2.4.1. Обработка датасета с Bird Annotation

In [19]:
dataBirdAnnotation = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_BIRD_ANNOTATION}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_BIRD_ANNOTATION}{sample}/labels/', 3)) # Получаем датасет с разметкой
    dataBirdAnnotation  = pd.concat([dataBirdAnnotation, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataBirdAnnotation = datasetMinimization(dataBirdAnnotation, DATASET_DOWNGRADES[print_variable_name(dataBirdAnnotation)])

dataBirdAnnotation.sample(10) # Посмотрим на данные

  0%|          | 0/1362 [00:00<?, ?it/s]

  0%|          | 0/130 [00:00<?, ?it/s]

  0%|          | 0/65 [00:00<?, ?it/s]

There were 1557 photos, now there are 389


,image_name,x_center,y_center,w,h,classes
15273,./DATASET/BirdAnnotation/train/images/voladora...,0.171875,0.389844,0.06875,0.110937,3
5901,./DATASET/BirdAnnotation/train/images/pajarosc...,0.702344,0.295312,0.042188,0.05,3
5457,./DATASET/BirdAnnotation/train/images/pajarosc...,0.217969,0.810156,0.039062,0.05,3
15746,./DATASET/BirdAnnotation/train/images/voladora...,0.392969,0.635156,0.101562,0.048438,3
2935,./DATASET/BirdAnnotation/train/images/e221_jpg...,0.86875,0.40625,0.028125,0.04375,3
15465,./DATASET/BirdAnnotation/train/images/voladora...,0.873437,0.926562,0.05,0.034375,3
16603,./DATASET/BirdAnnotation/train/images/voladora...,0.128906,0.246094,0.129688,0.132812,3
760,./DATASET/BirdAnnotation/valid/images/voladora...,0.619531,0.522656,0.057813,0.029687,3
3737,./DATASET/BirdAnnotation/train/images/e241_jpg...,0.167187,0.601562,0.01875,0.025,3
5828,./DATASET/BirdAnnotation/train/images/pajarosc...,0.960156,0.414062,0.042188,0.042188,3


### 2.4.2. Обработка датасета с Birds Detection

In [20]:
dataBirdsDetectod = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_BIRDS_DETECTOR}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_BIRDS_DETECTOR}{sample}/labels/', 3)) # Получаем датасет с разметкой
    dataBirdsDetectod = pd.concat([dataBirdsDetectod, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataBirdsDetectod = datasetMinimization(dataBirdsDetectod, DATASET_DOWNGRADES[print_variable_name(dataBirdsDetectod)])

dataBirdsDetectod.sample(10)  # Посмотрим на данные

  0%|          | 0/594 [00:00<?, ?it/s]

  0%|          | 0/170 [00:00<?, ?it/s]

  0%|          | 0/85 [00:00<?, ?it/s]

There were 849 photos, now there are 212


,image_name,x_center,y_center,w,h,classes
2328,./DATASET/BirdsDetector/train/images/voladoras...,0.675781,0.80625,0.054688,0.04375,3
1653,./DATASET/BirdsDetector/train/images/gg37_jpg....,0.3,0.757031,0.083594,0.0875,3
92,./DATASET/BirdsDetector/train/images/51663_jpg...,0.5125,0.729688,0.0875,0.057031,3
2003,./DATASET/BirdsDetector/train/images/pajarosca...,0.636719,0.576562,0.628906,0.628125,3
15,./DATASET/BirdsDetector/train/images/-de-pajar...,0.373437,0.279687,0.092969,0.101562,3
3968,./DATASET/BirdsDetector/train/images/voladoras...,0.514844,0.217188,0.516406,0.339062,3
1588,./DATASET/BirdsDetector/train/images/gg28_jpg....,0.966406,0.785937,0.026562,0.089063,3
3767,./DATASET/BirdsDetector/train/images/voladoras...,0.546875,0.357031,0.073438,0.11875,3
2787,./DATASET/BirdsDetector/train/images/voladoras...,0.542188,0.475781,0.025781,0.071094,3
88,./DATASET/BirdsDetector/valid/images/basile-11...,0.34375,0.43125,0.5,0.542969,3


### 2.4.3. Обработка датасета с BirdsV3

In [21]:
dataBirdsV3 = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_BIRDS_V3}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_BIRDS_V3}{sample}/labels/', 3)) # Получаем датасет с разметкой
    dataBirdsV3 = pd.concat([dataBirdsV3, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataBirdsV3 = datasetMinimization(dataBirdsV3, DATASET_DOWNGRADES[print_variable_name(dataBirdsV3)])

dataBirdsV3.sample(10)  # Посмотрим на данные

  0%|          | 0/582 [00:00<?, ?it/s]

  0%|          | 0/55 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

There were 665 photos, now there are 332


,image_name,x_center,y_center,w,h,classes
3916,./DATASET/BirdsV3/train/images/pigeons-roof-fl...,0.076563,0.357031,0.05,0.045312,3
1859,./DATASET/BirdsV3/train/images/birds-roof-1271...,0.777344,0.589063,0.05,0.054688,3
1830,./DATASET/BirdsV3/train/images/birds-roof-1271...,0.5875,0.395313,0.035937,0.039062,3
746,./DATASET/BirdsV3/train/images/48708475716_d17...,0.170313,0.298438,0.220312,0.271875,3
2491,./DATASET/BirdsV3/train/images/DSC_0597_jpg.rf...,0.182031,0.360156,0.020313,0.029687,3
307,./DATASET/BirdsV3/train/images/2729102735_c620...,0.247656,0.298438,0.020313,0.034375,3
1188,./DATASET/BirdsV3/train/images/919183287_a475f...,0.282813,0.50625,0.1,0.165625,3
286,./DATASET/BirdsV3/train/images/2729102735_c620...,0.670312,0.45,0.023438,0.034375,3
161,./DATASET/BirdsV3/train/images/1_jpg.rf.7b6c8d...,0.010937,0.757031,0.021875,0.085938,3
1796,./DATASET/BirdsV3/train/images/birds-on-roof_j...,0.516406,0.807813,0.009375,0.039062,3


## 2.5. Обработка фотографий со зданиями
### 2.5.1. Обработка датасета Building Detection 

In [22]:
dataBuildingDetection = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_BUILDING_DETECTION}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_BUILDING_DETECTION}{sample}/labels/', 4)) # Получаем датасет с разметкой
    dataBuildingDetection = pd.concat([dataBuildingDetection, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataBuildingDetection = datasetMinimization(dataBuildingDetection, DATASET_DOWNGRADES[print_variable_name(dataBuildingDetection)])

dataBuildingDetection.sample(10)  # Посмотрим на данные

  0%|          | 0/726 [00:00<?, ?it/s]

  0%|          | 0/47 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

There were 802 photos, now there are 401


,image_name,x_center,y_center,w,h,classes
448,./DATASET/BuildingDetection/train/images/287_A...,0.5,0.498798,1.0,0.997596,4
315,./DATASET/BuildingDetection/train/images/1_A_j...,0.5,0.498798,1.0,0.997596,4
26,./DATASET/BuildingDetection/train/images/105_j...,0.5,0.498798,1.0,0.997596,4
519,./DATASET/BuildingDetection/train/images/307_A...,0.65625,0.507212,0.576923,0.980769,4
221,./DATASET/BuildingDetection/train/images/16_A_...,0.507212,0.521635,0.980769,0.951923,4
41,./DATASET/BuildingDetection/valid/images/45_A_...,0.591346,0.509615,0.800481,0.972356,4
15,./DATASET/BuildingDetection/test/images/188_A_...,0.444712,0.658654,0.586538,0.675481,4
92,./DATASET/BuildingDetection/train/images/128_A...,0.533079,0.535196,0.824567,0.922786,4
326,./DATASET/BuildingDetection/train/images/203_A...,0.5,0.498798,1.0,0.997596,4
106,./DATASET/BuildingDetection/train/images/131_A...,0.498798,0.498798,0.997596,0.997596,4


### 2.5.2. Обработка датасета Josh Edits 

In [23]:
dataJoshEdits = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_JOSH_EDITS}{sample}/labels') # Список файлов с разметкой 
        
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_JOSH_EDITS}{sample}/labels/', 4)) # Получаем датасет с разметкой
    dataJoshEdits = pd.concat([dataJoshEdits, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataJoshEdits = datasetMinimization(dataJoshEdits, DATASET_DOWNGRADES[print_variable_name(dataJoshEdits)])

dataJoshEdits.sample(10)  # Посмотрим на данные

  0%|          | 0/3036 [00:00<?, ?it/s]

  0%|          | 0/290 [00:00<?, ?it/s]

  0%|          | 0/139 [00:00<?, ?it/s]

There were 3251 photos, now there are 1625


,image_name,x_center,y_center,w,h,classes
2268,./DATASET/JoshEdits/train/images/image_58_png....,0.858594,0.677344,0.282813,0.445312,4
4312,./DATASET/JoshEdits/train/images/sv_37-4270221...,0.857031,0.319531,0.285938,0.517188,4
1503,./DATASET/JoshEdits/train/images/debug_image_2...,0.403906,0.514844,0.620313,0.176563,4
1485,./DATASET/JoshEdits/train/images/debug_image_2...,0.894531,0.553906,0.169531,0.096875,4
4114,./DATASET/JoshEdits/train/images/sv_37-4088640...,0.125781,0.475781,0.251563,0.30625,4
2051,./DATASET/JoshEdits/train/images/image_22_png....,0.057813,0.625781,0.115625,0.539062,4
484,./DATASET/JoshEdits/train/images/37_4036787381...,0.978906,0.488281,0.042188,0.071094,4
2679,./DATASET/JoshEdits/train/images/modeldec_38-1...,0.129688,0.525,0.259375,0.165625,4
2812,./DATASET/JoshEdits/train/images/modelok_36-69...,0.882031,0.385156,0.235937,0.254688,4
849,./DATASET/JoshEdits/train/images/Abandoned-Cle...,0.585156,0.477344,0.759375,0.884375,4


# 3. Объединение датасетов в единый датасет

In [24]:
data = pd.concat(
    [
        dataCarDetection, dataCarPerson, dataCarsDetection, # Авто
        dataSTN, dataUnilityPole, dataTransmission, # ЛЭП
        dataTree, dataTreeDetection, dataTree100, # Деревья
        dataBirdAnnotation, dataBirdsDetectod, dataBirdsV3, # Птицы
        dataBuildingDetection, dataJoshEdits # Здания
    ], 
    ignore_index=True
) # Объеденим датасеты в один единый

del dataCarDetection, dataCarPerson, dataCarsDetection # очищаем память от датасетов с Авто
del dataSTN, dataUnilityPole, dataTransmission # очищаем память от датасетов с ЛЭП
del dataTree, dataTreeDetection, dataTree100 # очищаем память от датасетов со Деревьями
del dataBirdAnnotation, dataBirdsDetectod, dataBirdsV3 # очищаем память от датасетов со Птицами
del dataBuildingDetection, dataJoshEdits # очищаем память от датасетов со Зданиями

data.sample(10) # Посмотрим на данные

,image_name,x_center,y_center,w,h,classes
25699,./DATASET/BirdAnnotation/valid/images/voladora...,0.134375,0.675781,0.03125,0.017188,3
16073,./DATASET/Transmission/data_original_size_v1/d...,0.327083,0.327083,0.120833,0.523457,5
2512,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.09375,0.780048,0.186298,0.183894,0
9181,./DATASET/Transmission/data_original_size_v1/d...,0.632767,0.632767,0.7322,0.28668,5
10804,./DATASET/Transmission/data_original_size_v1/d...,0.49987,0.49987,0.99974,0.476221,5
21990,./DATASET/BirdAnnotation/train/images/voladora...,0.54375,0.859375,0.028125,0.059375,3
30825,./DATASET/JoshEdits/train/images/c039227312edd...,0.182031,0.510938,0.364063,0.934375,4
22901,./DATASET/BirdAnnotation/train/images/voladora...,0.150781,0.522656,0.032813,0.053125,3
14853,./DATASET/Transmission/data_original_size_v1/d...,0.493908,0.493908,0.987816,0.999537,5
7853,./DATASET/Transmission/data_original_size_v1/d...,0.49987,0.49987,0.99974,0.199373,5


In [25]:
data['image_name'].unique().shape # Количество фотографий

(7105,)

In [26]:
# Изменим тип данных координат с string на float
for column in data.columns[1:-1]:
    data[column] = data[column].astype(float)

data['classes'] = data['classes'].astype(int) # Изменим тип классов с string на int
data.info() # Посмотрим на сведение дата сета

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33643 entries, 0 to 33642
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   image_name  33643 non-null  object 
 1   x_center    33643 non-null  float64
 2   y_center    33643 non-null  float64
 3   w           33643 non-null  float64
 4   h           33643 non-null  float64
 5   classes     33643 non-null  int32  
dtypes: float64(4), int32(1), object(1)
memory usage: 1.4+ MB


In [27]:
count_image = data['image_name'].unique().shape[0]
print(f'Dataset contains {count_image} unique photos and {data.shape[0]} objects')
del count_image # Очищаем память

Dataset contains 7105 unique photos and 33643 objects


    0 - машина
    1 - ЛЭП
    2 - Деревья
    3 - Птицы
    4 - Дом/здание
    5 - Провода

In [28]:
# Выведем в процентах количество уникальных изображений по классам
data.groupby('image_name')['classes'].head().value_counts().sort_index() /  data.groupby('image_name')['classes'].head().shape[0] * 100

classes
0    21.150288
1     3.966699
2    13.815431
3    14.892807
4    20.100120
5    26.074654
Name: count, dtype: float64

    0 - машина
    1 - ЛЭП
    2 - Деревья
    3 - Птицы
    4 - Дом/здание
    5 - Провода

In [29]:
# Распределение классов среди всех объектов
data['classes'].value_counts().sort_index() / data.shape[0] * 100

classes
0    18.428796
1     2.612728
2     8.239455
3    29.756562
4    11.015664
5    29.946794
Name: count, dtype: float64

In [30]:
# Разделим данные на выборки
train_name_file, test_name_file = train_test_split(data['image_name'].unique(), test_size=.30, random_state=42) 
valid_name_file, test_name_file = train_test_split(test_name_file, test_size=.33, random_state=42)

# Посмотрим на процентное отношение выборок 
pd.DataFrame([len(train_name_file), len(valid_name_file), len(test_name_file)], columns=['size']) / len(data['image_name'].unique()) * 100

,size
0,69.992963
1,20.098522
2,9.908515


In [31]:
train_dataCarDetection = data[data['image_name'].isin(train_name_file)] # Записи для тренировочных данных
valid_dataCarDetection = data[data['image_name'].isin(valid_name_file)] # Записи для валидационных данных
test_dataCarDetection = data[data['image_name'].isin(test_name_file)] # Записи для тестовых данных
del data, train_name_file, valid_name_file, test_name_file # Очистим память

In [32]:
import os
list_paths_to_create = [
    'DATA', 'DATA/train', 
    'DATA/valid', 'DATA/test', 'DATA/train/images', 
    'DATA/train/labels', 'DATA/valid/images', 'DATA/valid/labels',
    'DATA/test/images', 'DATA/test/labels'
]

for i in list_paths_to_create:
    os.mkdir(i)


In [33]:
def copyFileDataSet(dataCarDetection, pathEnd):
    '''
        Функция позволяет скопировать фото в новую папку и под каждую из них создать txt файл с разметкой
        Input:
            dataCarDetection - DataFrame содержащий в себе следующиго типа:
                Признаки    Non-Null?  Тип
                image_name  non-null   object 
                x_center    non-null   float64
                y_center    non-null   float64
                w           non-null   float64
                h           non-null   float64
                classes     non-null   int64
            pathEnd - путь созданной папки, внутри которой созданы папки
                images и labels
    '''
    nameFiles = dataCarDetection['image_name'].unique() # Получаем список уникальных файлов

    # Запускаем цикл по все файлам
    for item in tqdm(nameFiles):

        # shutil.copy2(item, pathEnd+'/images') # Копируем данный файл в другую директорию/
        img = cv2.imread(item, cv2.IMREAD_UNCHANGED)
        img = cv2.resize(cv2.UMat(img), (1280, 720), cv2.INTER_NEAREST)
        item = item.replace('\\', '/')
        pathEnd = pathEnd.replace('\\', '/')
        
        cv2.imwrite(pathEnd + '/images/' + item.split('/')[-1], img)
        
        dataList = list(dataCarDetection[dataCarDetection['image_name'] == item].values) # Получаем все объекты на фото
        labelsFile = open(pathEnd + '/labels/' + item.split('/')[-1][:-3] + 'txt', 'w+') # Создаём текстовый документ с разметкой
        isFirst = True # Флаг первой строки

        # Запускаем цикл по всем объектам на фото
        for data in dataList:

            # Получаем линию с данными 
            line = '' if isFirst else '\n'
            isFirst = False
            line += str(data[-1]) + ' ' + ' '.join(str(number) for number in data[1:-1]) 
            
            labelsFile.write(line) # Записываем в файл
        labelsFile.close() # Закрываем файл
        
copyFileDataSet(train_dataCarDetection, './DATA/train') # Формируем тренировочную выборку
copyFileDataSet(valid_dataCarDetection, './DATA/valid') # Формируем валидационную выборку
copyFileDataSet(test_dataCarDetection, './DATA/test') # Формируем тестовую выборку
del train_dataCarDetection, valid_dataCarDetection, test_dataCarDetection # Очищаем память

  0%|          | 0/4973 [00:00<?, ?it/s]

./DATASET/CarDetection/training_images/vid_4_1000.jpg
./DATASET/CarDetection/training_images/vid_4_10000.jpg
./DATASET/CarDetection/training_images/vid_4_10020.jpg
./DATASET/CarDetection/training_images/vid_4_10060.jpg
./DATASET/CarDetection/training_images/vid_4_10100.jpg
./DATASET/CarDetection/training_images/vid_4_10120.jpg
./DATASET/CarDetection/training_images/vid_4_10480.jpg
./DATASET/CarDetection/training_images/vid_4_10500.jpg
./DATASET/CarDetection/training_images/vid_4_1060.jpg
./DATASET/CarDetection/training_images/vid_4_10960.jpg
./DATASET/CarDetection/training_images/vid_4_11240.jpg
./DATASET/CarDetection/training_images/vid_4_11380.jpg
./DATASET/CarDetection/training_images/vid_4_11400.jpg
./DATASET/CarDetection/training_images/vid_4_11880.jpg
./DATASET/CarDetection/training_images/vid_4_12100.jpg
./DATASET/CarDetection/training_images/vid_4_12120.jpg
./DATASET/CarDetection/training_images/vid_4_12180.jpg
./DATASET/CarDetection/training_images/vid_4_12260.jpg
./DATASET/Ca

  0%|          | 0/1428 [00:00<?, ?it/s]

./DATASET/CarDetection/training_images/vid_4_10520.jpg
./DATASET/CarDetection/training_images/vid_4_11440.jpg
./DATASET/CarDetection/training_images/vid_4_11920.jpg
./DATASET/CarDetection/training_images/vid_4_12360.jpg
./DATASET/CarDetection/training_images/vid_4_13680.jpg
./DATASET/CarDetection/training_images/vid_4_13720.jpg
./DATASET/CarDetection/training_images/vid_4_13800.jpg
./DATASET/CarDetection/training_images/vid_4_14140.jpg
./DATASET/CarDetection/training_images/vid_4_16060.jpg
./DATASET/CarDetection/training_images/vid_4_16660.jpg
./DATASET/CarDetection/training_images/vid_4_1760.jpg
./DATASET/CarDetection/training_images/vid_4_1820.jpg
./DATASET/CarDetection/training_images/vid_4_1920.jpg
./DATASET/CarDetection/training_images/vid_4_19780.jpg
./DATASET/CarDetection/training_images/vid_4_2080.jpg
./DATASET/CarDetection/training_images/vid_4_21180.jpg
./DATASET/CarDetection/training_images/vid_4_21420.jpg
./DATASET/CarDetection/training_images/vid_4_21560.jpg
./DATASET/CarD

  0%|          | 0/704 [00:00<?, ?it/s]

./DATASET/CarDetection/training_images/vid_4_11420.jpg
./DATASET/CarDetection/training_images/vid_4_12080.jpg
./DATASET/CarDetection/training_images/vid_4_12280.jpg
./DATASET/CarDetection/training_images/vid_4_14500.jpg
./DATASET/CarDetection/training_images/vid_4_16140.jpg
./DATASET/CarDetection/training_images/vid_4_16700.jpg
./DATASET/CarDetection/training_images/vid_4_17160.jpg
./DATASET/CarDetection/training_images/vid_4_17440.jpg
./DATASET/CarDetection/training_images/vid_4_1840.jpg
./DATASET/CarDetection/training_images/vid_4_1900.jpg
./DATASET/CarDetection/training_images/vid_4_21520.jpg
./DATASET/CarDetection/training_images/vid_4_26440.jpg
./DATASET/CarDetection/training_images/vid_4_29940.jpg
./DATASET/CarDetection/training_images/vid_4_3460.jpg
./DATASET/CarDetection/training_images/vid_4_6420.jpg
./DATASET/CarDetection/training_images/vid_4_8280.jpg
./DATASET/CarDetection/training_images/vid_4_9460.jpg
./DATASET/CarPerson/Car-Person-v2-Roboflow-Owais-Ahmad/train/images/car

In [34]:
# Глобальный путь до директории с проектом 
global_path = os.path.abspath('.') 
yolo_format = dict(
    path=f'{global_path}/DATA',
    train='../train',
    val='../valid',
    test='../test',       
    nc=6,
    names={
        0: 'Car',
        1: 'STN',
        2: 'Tree',
        3: 'Bird',
        4: 'House',
        5: 'Cable',
    }
)

del global_path # Очищаем память
             
with open('./DATA/yolo.yaml', 'w') as outfile:
    yaml.dump(yolo_format, outfile, default_flow_style=False)

In [35]:
epochTraning = 250 # Количество эпох обучения
pathYalm = "./DATA/yolo.yaml" # Путь до файла yalm
patience = 10 # Терпимость эпох для переобучения
imgsz = 1280 # Размер изображения

In [36]:
model = YOLO('yolov8n.yaml').load('yolov8n.pt')
model.train(
    data=pathYalm, 
    epochs=epochTraning, 
    patience=patience, 
    imgsz=imgsz
)  

KeyboardInterrupt: 